In [1]:
% matplotlib inline
% load_ext autoreload
% autoreload 2


In [2]:
from utils import *
import nltk
from nltk.tokenize import word_tokenize
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from torch import nn
from torch.nn import Module
from torch.autograd import backward,Variable
from torch.optim import lr_scheduler, Adam
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms

In [3]:
words_to_index, index_to_words, word_to_vec_map = read_glove_vecs()

In [4]:
training_data = np.load('data/train.npy')
print('training data', training_data.shape[0])
sentences = training_data[:, 0]
labels = np.array(list(map(int,training_data[:, 1])))


training data 5573


In [5]:
cnt = 0
maxlen=0
sentence: str
for sentence in training_data[:, 0]:
    words=word_tokenize(sentence)
    maxlen=max(maxlen,len(words))
    for word in words:
        if word.lower() not in words_to_index.keys():
            cnt += 1
print(cnt,'unknowns')
print('maxlen=',maxlen)

6478 unknowns
maxlen= 219


In [6]:
def sentences_to_indices():
    result = np.zeros((len(training_data), maxlen),dtype=np.int32)
    for i, sentence in enumerate(sentences):
        for j, word in enumerate(word_tokenize(sentence)):
            word = word.lower()
            if word in words_to_index.keys():
                result[i, j] = words_to_index[word]
            else:
                result[i, j] = len(index_to_words)+1
    return result

In [7]:
indices=sentences_to_indices()
print(indices.shape)

(5573, 219)


In [8]:
# GRADED FUNCTION: pretrained_embedding_layer
from keras.layers import Embedding

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 2                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len,emb_dim))
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len,emb_dim)
    embedding_layer.trainable=False
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

/home/linghao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# GRADED FUNCTION: Emojify_V2
from keras import Input, Model
from keras.layers import LSTM, Dropout, Dense, Activation

def Emojify_V2(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(shape=input_shape,dtype='int32')
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128,return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(2,activation='softmax')(X)
    # Add a softmax activation
    X = Activation(activation='softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(sentence_indices,X)
    
    ### END CODE HERE ###
    
    return model

In [10]:
model = Emojify_V2((maxlen,), word_to_vec_map, words_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 219)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 219, 50)           20000100  
_________________________________________________________________
lstm_1 (LSTM)                (None, 219, 128)          91648     
_________________________________________________________________
dropout_1 (Dropout)          (None, 219, 128)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
__________

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
from keras.utils.np_utils import to_categorical


Epoch 1/50


 32/100 [========>.....................] - ETA: 0s - loss: 0.4695 - acc: 0.8438

 64/100 [==================>...........] - ETA: 0s - loss: 0.4539 - acc: 0.8594

 96/100 [===========================>..] - ETA: 0s - loss: 0.4903 - acc: 0.8229

100/100 [==============================] - 2s 18ms/step - loss: 0.4833 - acc: 0.8300


...

 96/100 [===========================>..] - ETA: 0s - loss: 0.4903 - acc: 0.8229

100/100 [==============================] - 2s 17ms/step - loss: 0.4833 - acc: 0.8300


Epoch 7/50


 32/100 [========>.....................] - ETA: 0s - loss: 0.4695 - acc: 0.8438

KeyboardInterrupt: 

In [14]:

model.fit(indices[0:100], to_categorical(labels[0:100], num_classes=2), epochs=50, batch_size=32, shuffle=True)


Epoch 1/50


 32/100 [========>.....................] - ETA: 0s - loss: 0.4695 - acc: 0.8438

 64/100 [==================>...........] - ETA: 0s - loss: 0.4539 - acc: 0.8594

 96/100 [===========================>..] - ETA: 0s - loss: 0.4903 - acc: 0.8229

100/100 [==============================] - 2s 18ms/step - loss: 0.4833 - acc: 0.8300


...

 96/100 [===========================>..] - ETA: 0s - loss: 0.4903 - acc: 0.8229

100/100 [==============================] - 2s 17ms/step - loss: 0.4833 - acc: 0.8300


Epoch 7/50


 32/100 [========>.....................] - ETA: 0s - loss: 0.4695 - acc: 0.8438

KeyboardInterrupt: 